In [43]:
import sys, pygame
from pygame import Color, surfarray
import numpy as np
import random
import cv2
import os

In [144]:
display_width = 900
display_height = 900

cell_size = 20 # Number of pixels per cell

cell_width = display_width//cell_size
cell_height = display_height//cell_size

display_width = cell_width*cell_size
display_height = cell_height*cell_size

INIT_MODE = 'subrandom' # 'subrandom'=random sized box smaller than cell width, 'random'=random 0 or 1 for each cell in row
EVOLVE_COLOR = False # Randomly delta color on each new row
SAVE_TO_DISK = False # Save each iteration as image in Screenshots/
EMPTY_RESET = False # Reset if new rows are almost entirely empty
ITERATION_CNT = 500 # Number of cellular evolutions 

rules_live = [0, 0, 1, 1, 0, 0, 0, 0, 0] # Index is neighbor living state count
rules_dead = [0, 0, 0, 1, 0, 0, 0, 0, 0] # Index is neighbor living state count

In [149]:
def init_states(itype='random'):
    if itype == 'random':
        states = np.random.randint(2, size=(cell_width, cell_height))
    elif itype == 'subrandom':
        randstart = random.randrange(0, cell_width//2)
        randstop = random.randrange(randstart, cell_width)
        states = np.zeros((cell_width, cell_height))
        states[randstart:randstop, randstart:randstop] = np.random.randint(2, size=(randstop-randstart, randstop-randstart))
    else:
        states = np.ones((cell_width, cell_height))
        states[0:cell_width//2, 0:cell_width//2] = np.ones((cell_width//2, cell_width//2))
    return states

In [216]:
"""
    Returns single cell result using defined rules and input region of interest
    - roi: region of interest. centered on x, y with padding region on cells > 0
    
    Note: currently only capable of 1 cell padding
"""
def next_gen(roi):
    live = roi[roi.shape[0]//2, roi.shape[1]//2] # Cell State
    live_cnt = int(np.sum(roi) - live) # don't count center cell
    
    assert len(rules_live) == roi.shape[0]*roi.shape[1], "Region of interest size does not match rules_live length"
    assert len(rules_dead) == roi.shape[0]*roi.shape[1], "Region of interest size does not match rules_dead length"
           
    try:    
        if live:
            return rules_live[live_cnt]
        else:
            return rules_dead[live_cnt]
    except:
        print(live_cnt, live)
        sys.exit()


def new_state(states, colors, pad=1):
    for y in range(pad, states.shape[0] - pad):
        for x in range(pad, states.shape[1] - pad):
            ng = next_gen(states[y-pad:y+pad+1, x-pad:x+pad+1]) # Next Generation Cell Value (int)
            
            states[y, x] = ng
            
            # Update Color
            dead_color=(255, 255, 255)
            color = get_avg_color(colors[y-pad:y+pad+1, x-pad:x+pad+1])
            colors[y, x] = update_color(ng, color, mode='random', dead_color=dead_color, max_delta=50)
    
    return states # return state array 

In [218]:
def get_avg_color(colors):
    colors = np.array(np.nanmean(np.where(colors != 255, colors, np.nan), axis=(0, 1)), dtype=int)
    return colors
    #return np.mean(colors, axis=(0, 1), dtype=int)

In [219]:
get_avg_color(px[10:15, 10:15])

array([40, 47, 23])

In [220]:
"""
    Pass in an RGB tuple. If mode=random, randomly increment or decrement the color value (R, G, or B randomly chosen).
    - live [int]: cell alive or dead
    - color: RGB tuple
    - max_delta: max amount to randomly increment or decrement the color scaler value
    - mode: random or always 255
"""
def update_color(live, color, max_delta=40, mode='random', dead_color=(255, 255, 255)):
    if not(live):
        return dead_color # Dead color
    
    color = list(color)
    if mode == 'random':
        rgb = random.randrange(3) # Random Channel
        delta = random.randrange(-max_delta, max_delta) # Random change in color

        new_color = color[rgb] + delta
        
        if new_color < 0:
            new_color = 0
        elif new_color > 255:
            new_color = 255
            
        color[rgb] = new_color
    else:
        color = (0, 0, 0)
        
    return tuple(color)

In [258]:
def init_colors(colors, live_color=(0,0,0)):
    for y in range(colors.shape[0]):
        for x in range(colors.shape[1]):
            color = colors[y, x]
            
            if all([a == b for a, b in zip(color, live_color)]):
                colors[y, x] = (random.randrange(0, 255), random.randrange(0, 255), random.randrange(0, 255))
                
    return colors

In [259]:
def save_frame(img, cnt):
    fname = format(f'{cnt:08d}.jpg')
    img = np.array(img)
    img = np.rot90(img, 3)
    img = np.flip(img, axis=1)
    img = cv2.imwrite(os.path.join('Screenshots', fname), img)
    
def save_screenshots_to_video(filename='1D_Cellular_Automata.avi', framerate=60, startframe=30, stopframe=300):
    video = []
    for i, file in enumerate(os.listdir('Screenshots')):
        if i < startframe:
            continue
        elif i > stopframe:
            break
        
        if file.endswith('.jpg'):
            img = cv2.imread(os.path.join('Screenshots', file))
            height, width, layers = img.shape
            size = (width, height)
            video += [img]

    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'DIVX'), framerate, size)
    for frame in video:
        out.write(frame)
    out.release()

In [260]:
states = init_states(itype=INIT_MODE) # Initialize States

pygame.init()

screen = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('2D Cellular Automata')

#create a surface with the size as the array
surface = pygame.Surface((cell_width, cell_height))
surface = pygame.transform.scale(surface, (display_width, display_height)) # Scale Size Up

# Initialize screen to white
px = pygame.surfarray.pixels3d(surface)
px[:,:,:] = np.ones((display_width, display_height, 3))*255 


run = True
cnt = 0
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False

    pygame.time.delay(30)
    
    # Reset bottom row if limited complexity
    if sum(states[0])/len(states[0]) < 0.05 and EMPTY_RESET:
        states[0] = init_states(return_btm=True)
    
    # Scale Screen Size Down to edit pixel array at cell size=1 --------------------------------------------------
    surface = pygame.transform.scale(surface, (cell_width, cell_height))
    px = pygame.surfarray.pixels3d(surface)
    
    # Iterate State
    states = new_state(states, px)
    
    if cnt == 0:
        init_colors(px)
    
    # Scale Screen Size Up to display pixel array at cell size=multiplier ----------------------------------------
    surface = pygame.transform.scale(surface, (display_width, display_height))  # Scale Size Up     
    screen.blit(surface, (0, 0)) # Update all pixels on screen object
    
    if SAVE_TO_DISK:
        save_frame(surfarray.array3d(surface), cnt)
    
    pygame.display.update()
        
    if cnt > ITERATION_CNT:
        break
    cnt += 1
    
pygame.quit()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in long_scalars


[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]


In [154]:
save_screenshots_to_video(filename='gif.avi', framerate=30, startframe=90, stopframe=300) # Run if screenshots already saved to disk

In [162]:
(0, 255, 0) == (0,233,0)

False

In [163]:
print(color)

NameError: name 'color' is not defined

In [175]:
colors

UsageError: %colors: you must specify a color scheme. See '%colors?'


In [178]:
test = px[10:15, 10:15]